# Merge transcript ticker and csr ticker

In [2]:
import pandas as pd

In [3]:
wrds_tickers = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CapitalIQ_Transcript_WRDS_Data/additional_reference_tables/wrds_ticker.csv")
wrds_tickers.head()


/tmp/ipykernel_936310/1474921117.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  wrds_tickers = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CapitalIQ_Transcript_WRDS_Data/additional_reference_tables/wrds_ticker.csv")


,companyid,ticker,startdate,enddate,companyname
0,18507.0,1026283,2000-02-15,2004-12-16,2M Invest A/S
1,18507.0,1026852,2005-01-29,2007-09-15,2M Invest A/S
2,18507.0,MMINV,2000-02-15,2004-12-16,2M Invest A/S
3,18507.0,MMINVN,2005-01-29,2005-10-29,2M Invest A/S
4,18507.0,TW8,2000-03-24,2004-12-16,2M Invest A/S


In [4]:
csr_ticker = pd.read_csv("/opt/hdd_1/research_hub/csr_project/CSR Reporting/csr_ticker_name.csv")
csr_ticker.head()

,ticker,COMPANY
0,A,Agilent Technologies Inc.
1,AA,Alcoa Inc.
2,AAL,American Airlines Group
3,AAON,AAON Inc.
4,AAP,Advance Auto Parts Inc.


In [6]:
# merge wrds_tickers (ticker, companyname) to csr_ticker on ticker
merged_tickers = pd.merge(csr_ticker, wrds_tickers, how='left', left_on='ticker', right_on='ticker')
merged_tickers = merged_tickers.rename(columns={"companyname": "wrds_companyname"})

In [7]:
merged_tickers

,ticker,COMPANY,companyid,startdate,enddate,wrds_companyname
0,A,Agilent Technologies Inc.,25228.0,2003-10-28,2003-10-28,Attwoods plc
1,A,Agilent Technologies Inc.,154924.0,2002-07-13,NaN,"Agilent Technologies, Inc."
2,A,Agilent Technologies Inc.,154924.0,2020-11-19,2022-10-04,"Agilent Technologies, Inc."
3,A,Agilent Technologies Inc.,383605.0,2010-07-19,2010-07-19,AstraZeneca AB
4,A,Agilent Technologies Inc.,527596.0,2002-07-13,2007-10-30,Resolute FP Canada Inc.
...,...,...,...,...,...,...
9386,ZMH,Zimmer Holdings Inc.,20630255.0,2008-08-21,NaN,Z.M.H Hammerman Ltd
9387,ZNH,China Southern Airlines Company Limited,365056.0,2002-07-13,2023-02-03,China Southern Airlines Company Limited
9388,ZNH,China Southern Airlines Company Limited,365056.0,2005-01-29,2008-12-27,China Southern Airlines Company Limited
9389,ZNH,China Southern Airlines Company Limited,365056.0,2008-12-27,2023-03-08,China Southern Airlines Company Limited


In [9]:
from fuzzywuzzy import fuzz

/home/francia/anaconda3/envs/csr_env/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [10]:
def get_most_similar(group):
    target = group['COMPANY'].iloc[0]  # 取該 ticker 下的公司名稱
    group['similarity'] = group['wrds_companyname'].apply(lambda x: fuzz.ratio(str(x), str(target)))
    return group.loc[group['similarity'].idxmax()]  # 回傳相似度最高的一列

# Apply by ticker group
best_matches = merged_tickers.groupby('ticker', group_keys=False).apply(get_most_similar)

# 最終結果
best_matches = best_matches.drop(columns=['similarity'])  # 移除相似度欄
best_matches.reset_index(drop=True, inplace=True)

best_matches.head()

/tmp/ipykernel_936310/1894588256.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_matches = merged_tickers.groupby('ticker', group_keys=False).apply(get_most_similar)


,ticker,COMPANY,companyid,startdate,enddate,wrds_companyname
0,A,Agilent Technologies Inc.,154924.0,2002-07-13,NaN,"Agilent Technologies, Inc."
1,AA,Alcoa Inc.,249603.0,1999-01-04,2016-10-05,Alcoa Inc.
2,AAL,American Airlines Group,168569.0,2013-12-09,NaN,American Airlines Group Inc.
3,AAON,AAON Inc.,320500.0,2002-07-13,NaN,"AAON, Inc."
4,AAP,Advance Auto Parts Inc.,24333.0,2002-07-13,NaN,"Advance Auto Parts, Inc."


In [13]:
matches_all = best_matches.dropna(subset=['wrds_companyname'])
matches_all

,ticker,COMPANY,companyid,startdate,enddate,wrds_companyname
0,A,Agilent Technologies Inc.,154924.0,2002-07-13,NaN,"Agilent Technologies, Inc."
1,AA,Alcoa Inc.,249603.0,1999-01-04,2016-10-05,Alcoa Inc.
2,AAL,American Airlines Group,168569.0,2013-12-09,NaN,American Airlines Group Inc.
3,AAON,AAON Inc.,320500.0,2002-07-13,NaN,"AAON, Inc."
4,AAP,Advance Auto Parts Inc.,24333.0,2002-07-13,NaN,"Advance Auto Parts, Inc."
...,...,...,...,...,...,...
1144,ZEAL,Zealand Pharma A/S,1268514.0,2006-01-05,NaN,Zealand Pharma A/S
1145,ZEN,Zendesk Inc,46240937.0,2014-05-13,2022-11-21,"Zendesk, Inc."
1146,ZION,Zions Bancorporation,315476.0,2002-07-13,NaN,"Zions Bancorporation, National Association"
1147,ZMH,Zimmer Holdings Inc.,264048.0,2002-07-13,2015-06-30,"Zimmer Biomet Holdings, Inc."
